In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager


In [20]:
def scarpe(auction_id):
    path = f"https://www.centralcarauctions.com/auction/{auction_id}"
    options = ChromeOptions()
    options.add_argument("--start-maximized")
    # options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[text() = "Accept additional cookies"]')))
        if cookie:
            driver.execute_script("arguments[0].scrollIntoView();", cookie)
            cookie.click()
    except:
        pass
    
    # clck login
    try:
        # log_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            # driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
            time.sleep(2)
            for i in range(2):
                driver.back()
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
        
        
     # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="pagination-pages"]/span[last()]'))).text.strip("()")
        if cars:
            # print(cars)
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[0])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
        
    
    
    # result
    results = []
    car_count = 0
    while car_count < total_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}  
                    
                    
                    # car details complete
                    try:
                        det_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="order-lg-2 col-12 col-lg-5 auction-details"]'))) 
                        if det_card:
                            
                            # title and loc --> done
                            try:
                                title =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]'))).text.strip()
                                if title:
                                    details['Title'] = title.replace("\n", " ")
                                else:
                                    details['Title'] = 'na'
                                
                                make = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/h1'))).text.strip()
                                if make:
                                    details['make'] = make
                                else:
                                    details['make'] = 'na'
                                
                                model = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
                                if model:
                                    details['model'] = model.split(" ")[0]
                                else:
                                    details['model'] = 'na'
                                
                                variant = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
                                if variant:
                                    details['variant'] = " ".join(variant.split(" ")[1:])
                                else:
                                    details['variant'] = 'na'
                                
                                loc =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="auction-location"]'))).text.strip()
                                if loc:
                                    details['Center'] = loc
                                else:
                                    details['Center'] = 'na'
                                    
                            except Exception as e:
                                print("No veh title and location found", e)        

                            # main agent --> done
                            try:
                                agent = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]'))) 
                                if agent:
                                    elements = agent.find_elements(By.TAG_NAME, "p")
                                    if elements:
                                        for ele in elements:
                                            agent_lbl = ele.find_element(By.TAG_NAME, 'strong')
                                            agent_val = ele.text.replace(agent_lbl.text, "").strip()
                                            if agent_lbl and agent_val:
                                                details[agent_lbl.text.strip()] = agent_val
                                            else:
                                                print("No agent data found")
                                    else:
                                        print("No agent elements")
                                    
                                    lane =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]/p[3]')))     
                                    if lane:
                                        details['Lane'] = lane.text.strip().split(" ")[1]
                                    else:
                                        details['Lane'] = 'na'                             
                            except Exception as e:
                                print(" No agent card")
                                
                            # pricing --> done
                            try:
                                pricing_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-list"]'))) 
                                if pricing_card:
                                    pri_elements = pricing_card.find_elements(By.TAG_NAME, "p")
                                    if pri_elements:
                                        for pri_ele in pri_elements:
                                            pri_lbl = pri_ele.find_element(By.TAG_NAME, 'strong')
                                            pri_val = pri_ele.text.replace(pri_lbl.text, "").strip()
                                            if pri_lbl and pri_val:
                                                details[pri_lbl.text.strip()] = pri_val
                                            else:
                                                print("No pricing found")
                            except Exception as e:
                                print(" No pricing card")
                                    
                            
                            # pricing footer --> done
                            try:
                                footer_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-footer"]'))) 
                                if footer_card:
                                    fot_ele = footer_card.find_element(By.TAG_NAME, "p")
                                    if fot_ele:
                                        # for pri_ele in pri_elements:
                                        fot_lbl = fot_ele.find_element(By.TAG_NAME, 'strong')
                                        fot_val = fot_ele.text.replace(fot_lbl.text, "").strip()
                                        if fot_lbl and fot_val:
                                            details[fot_lbl.text.strip()] = fot_val
                                        else:
                                            print("No footer pricing found")
                            except Exception as e:
                                print(" No footer pricing card")
                            
                            # inspec rep --> done
                            try:
                                # Try with multiple possible patterns
                                inspec_link = None
                                possible_links = driver.find_elements(
                                    By.XPATH,
                                    './/a[contains(text(), "Download inspection report") or contains(@href, ".pdf")]'
                                )
                                for a in possible_links:
                                    href = a.get_attribute("href")
                                    if href and href.endswith(".pdf"):
                                        inspec_link = href
                                        break

                                if inspec_link:
                                    # full link bana lo agar relative ho
                                    if inspec_link.startswith("/"):
                                        inspec_link = "https://www.manheim.co.uk" + inspec_link
                                    details["Inspection Report"] = inspec_link
                                else:
                                    details["Inspection Report"] = "na"

                            except Exception as e:
                                print("No inspection report found:", e)
                                details["Inspection Report"] = "na"
                   
                    except Exception as e:
                        print("No main details card")
                    try:
                        non_runner_el = driver.find_element(
                            By.XPATH,
                            './/div[@class="page-title"]//h4//span[contains(text(), "Non Runner")]'
                        )
                        if non_runner_el:
                            details["Non Runner"] = "Non Runner"
                        else:
                            details["Non Runner"] = ""
                    except:
                        details["Non Runner"] = ""

                    try:
                        # Check for category badge like "Cat: S" or "Cat: N"
                        cat_badge = driver.find_element(
                            By.XPATH,
                            './/div[@class="page-title"]//h4//span[contains(text(), "Cat:")]'
                        ).text.strip()

                        # Check for detailed accident paragraph
                        try:
                            accident_detail = driver.find_element(
                                By.XPATH,
                                './/div[@class="page-title"]/p[contains(text(), "Category")]'
                            ).text.strip()
                        except:
                            accident_detail = ""

                        # Combine both into one clean field
                        if cat_badge or accident_detail:
                            details["Accident and Damage"] = ", ".join(
                                part for part in [cat_badge, accident_detail] if part
                            )
                        else:
                            details["Accident and Damage"] = ""
                    except:
                        details["Accident and Damage"] = ""
                    
                    try:
                        grade_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="gallery-main"]/img'))).get_attribute("src")
                        if grade_main:
                            grade = grade_main.split("/")[-1].split(".")[0]
                            details['Grade'] = grade
                        else:
                            details['Grade'] = 'na'
                    except Exception as e:
                        print("no grade")                            
                        
                            
                    # images --> done
                    try:
                        img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="ug-thumbs-strip-inner"]'))) 
                        if img_card:
                            imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
                            if imgs:
                                imgs_lst = [img.get_attribute("src") for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                            else:
                                print("No images")
                    except Exception as e:
                        print(f"No image card {e}")
                        
                    
                    # veh summary and inspection
                    try:
                        tabs_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@id="vehicleTab"]')))
                        if tabs_card:
                            
                            # veh summary --> done
                            try:
                                veh_sum_btn =WebDriverWait(tabs_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="summary-tab"]'))) 
                                if veh_sum_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", veh_sum_btn)
                                    veh_sum_btn.click()
                                    
                                    # veh summary
                                    try:  
                                        veh_cont_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="vehicleTabContent"]')))
                                        if veh_cont_card:
                                            # additional info --> done
                                            try:
                                                add_info = veh_cont_card.find_element(By.XPATH, './/div[@id="vehicleTabContent"]//p').text.strip()
                                                if add_info:
                                                    details['Additional Information'] = add_info
                                                else:
                                                    details['Additional Information'] = 'na'
                                            except Exception as e:
                                                print("No add info")
                                            
                                            # other info --> done
                                            try:       
                                                veh_sum = WebDriverWait(veh_cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-summary"]')))
                                                if veh_sum:
                                                    veh_trs = WebDriverWait(veh_sum, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                                    if veh_trs:
                                                        for veh_tr in veh_trs:
                                                            veh_lbl = veh_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                                            veh_val = veh_tr.find_element(By.TAG_NAME, 'strong').text.strip()
                                                            if veh_lbl and veh_val:
                                                                details[veh_lbl] = veh_val
                                                            else:
                                                                print("No veh details")
                                            except Exception as e:
                                                print("No veh summary info")        
                                        else:
                                            print("No veh summary button")
                                    except Exception as e:
                                        print("No veh summary card found")
                            except Exception as e:
                                print("No veh summary card")
                            
                            
                            # inspection --> remains
                            try:
                                inspec_btn = WebDriverWait(tabs_card, 5).until(
                                    EC.presence_of_element_located((By.XPATH, './/button[@id="spec-tab"]'))
                                )
                                if inspec_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", inspec_btn)
                                    inspec_btn.click()
                                    time.sleep(3)

                                    # --- Condition Images ---
                                    try:
                                        cond_imgs_card = WebDriverWait(driver, 5).until(
                                            EC.presence_of_element_located((By.XPATH, '(.//div[@class="slick-track"])[2]'))
                                        )
                                        if cond_imgs_card:
                                            cond_imgs = WebDriverWait(cond_imgs_card, 5).until(
                                                EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))
                                            )
                                            cond_imgs_lst = [
                                                cond_img.get_attribute("src") for cond_img in cond_imgs if cond_img.get_attribute("src")
                                            ]
                                            details["Damaged_images"] = ", ".join(cond_imgs_lst) if cond_imgs_lst else "na"
                                        else:
                                            print("no cond image card")
                                    except Exception:
                                        print("no cond image card")

                                    # --- Interior and Tyres ---
                                    try:
                                        int_tyre_card = WebDriverWait(driver, 5).until(
                                            EC.presence_of_element_located((By.XPATH, './/div[@class="row mb+"]'))
                                        )
                                        if int_tyre_card:
                                            # --- Interior / General Condition ---
                                            general_conditions = []
                                            try:
                                                inspec_rows = int_tyre_card.find_elements(
                                                    By.XPATH, '(.//div[@class="col-md-6"])[1]/table/tbody/tr'
                                                )
                                                for tr in inspec_rows:
                                                    lbl = tr.find_element(By.TAG_NAME, "th").text.strip()
                                                    val = tr.find_element(By.TAG_NAME, "td").text.strip()
                                                    # ❌ Skip agar value '-' ya blank ho
                                                    if lbl and val and val != "-":
                                                        general_conditions.append(f"{lbl}: {val}")
                                            except Exception:
                                                pass

                                            # --- Tyres Condition ---
                                            tyre_conditions = []
                                            try:
                                                tyre_rows = int_tyre_card.find_elements(
                                                    By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody/tr'
                                                )
                                                for tr in tyre_rows:
                                                    tds = [td.text.strip() for td in tr.find_elements(By.TAG_NAME, "td")]
                                                    if len(tds) >= 3:
                                                        tyre_name = tds[0]
                                                        cond1 = tds[1]
                                                        cond2 = tds[2]
                                                        # ❌ Skip agar sari values empty hain
                                                        if tyre_name and (cond1 or cond2):
                                                            # Remove empty commas
                                                            joined = ", ".join([v for v in [cond1, cond2] if v])
                                                            tyre_conditions.append(f"{tyre_name}: {joined}")
                                            except Exception:
                                                pass

                                            # ✅ Save only if data exists
                                            if general_conditions:
                                                details["General Condition"] = ", ".join(general_conditions)
                                            if tyre_conditions:
                                                details["Tyres Condition"] = ", ".join(tyre_conditions)

                                    except Exception as e:
                                        print("No interior and tyre data:", e)

                            except Exception:
                                print("No inspect tab")

                                
                    except Exception as e:
                        print("No buttons found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
        
                except Exception as e:
                    print("No cars found")        

            if car_count % 30 == 0: 
                try:
                    next_link = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Load More"]')))
                    if next_link:
                        time.sleep(5)
                        next_link.click()
                        
                        WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                        print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break
     
        except Exception as e:
            print("No cars found")
            break
        
    df = pd.DataFrame.from_dict(results)
    df.to_csv("data.csv", index=False)
    driver.quit()
# path = "https://www.centralcarauctions.com/auction/776"
scarpe(787)

no login
No username tab found and the error is Message: 
Stacktrace:
	GetHandleVerifier [0x0x3dfe43+66515]
	GetHandleVerifier [0x0x3dfe84+66580]
	(No symbol) [0x0x1cdc48]
	(No symbol) [0x0x218704]
	(No symbol) [0x0x218aab]
	(No symbol) [0x0x25f482]
	(No symbol) [0x0x23b214]
	(No symbol) [0x0x25cba7]
	(No symbol) [0x0x23afc6]
	(No symbol) [0x0x20c2ca]
	(No symbol) [0x0x20d154]
	GetHandleVerifier [0x0x637353+2521315]
	GetHandleVerifier [0x0x6322d3+2500707]
	GetHandleVerifier [0x0x407c94+229924]
	GetHandleVerifier [0x0x3f81f8+165768]
	GetHandleVerifier [0x0x3fecad+193085]
	GetHandleVerifier [0x0x3e8158+100072]
	GetHandleVerifier [0x0x3e82f0+100480]
	GetHandleVerifier [0x0x3d25aa+11066]
	BaseThreadInitThunk [0x0x759d7ba9+25]
	RtlInitializeExceptionChain [0x0x77ccc3ab+107]
	RtlClearBits [0x0x77ccc32f+191]
	(No symbol) [0x0]

No password tab found and the error is Message: 
Stacktrace:
	GetHandleVerifier [0x0x3dfe43+66515]
	GetHandleVerifier [0x0x3dfe84+66580]
	(No symbol) [0x0x1cdc48]
	(No

In [21]:
# def scarpe(path):
#     options = ChromeOptions()
#     options.headless=True
#     service = Service(ChromeDriverManager().install())
#     driver = Chrome(service=service, options=options)
#     driver.get(path)
#     driver.maximize_window()
    
    
#     try:
#         cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[text() = "Accept additional cookies"]')))
#         if cookie:
#             driver.execute_script("arguments[0].scrollIntoView();", cookie)
#             cookie.click()
#     except:
#         pass
    
#     # clck login
#     try:
#         # log_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
#         login = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
#         if login:
#             login.click()
#     except Exception as e:
#         print("no login")

#     # get the username tab
#     try:
#         provided_u_name = "fourbrotherstrading@icloud.com"
#         user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
#         user_name.send_keys(provided_u_name)
#     except Exception as e:
#         print(f"No username tab found and the error is {e}")
    
#     # get password tab
#     try:
#         provided_pass = "Sultanmirza1501#"
#         password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
#         driver.execute_script("arguments[0].scrollIntoView();", password)
#         password.send_keys(provided_pass)
#     except Exception as e:
#         print(f"No password tab found and the error is {e}") 
    
#     # get the check done
#     try:
#         check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
#         if check:
#             # driver.execute_script("arguments[0].scrollIntoView();", check)
#             check.click()
#             time.sleep(2)
#             for i in range(2):
#                 driver.back()
#     except Exception as e:
#         print(f"No check tab found and the error is {e}") 
        
        
#      # number of cars
#     try:
#         # time.sleep(1)
#         cars = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="pagination-pages"]/span[last()]'))).text.strip("()")
#         if cars:
#             # print(cars)
#             splited_cars = cars.split(" ")
#             total_cars = int(splited_cars[0])
#             print(f"{total_cars} number of cars found")
#         else:
#             print("No cars found")
#     except Exception as e:
#         print(f"No cars found and error is {e}")
        
    
    
#     # result
#     results = []
#     car_count = 0
#     while car_count < total_cars: #total_cars
        
#         try:
#             page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
            
#             for i in range(len(page_cars)): # len(page_cars)
#                 if car_count >= total_cars:
#                     break
                
#                 try:
#                     page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
#                     driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
#                     time.sleep(1)
#                     page_cars[i].click()

#                     details = {}  
                    
                    
#                     # car details complete
#                     try:
#                         det_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="order-lg-2 col-12 col-lg-5 auction-details"]'))) 
#                         if det_card:
                            
#                             # title and loc --> done
#                             try:
#                                 title =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]'))).text.strip()
#                                 if title:
#                                     details['Title'] = title.replace("\n", " ")
#                                 else:
#                                     details['Title'] = 'na'
                                
#                                 make = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/h1'))).text.strip()
#                                 if make:
#                                     details['make'] = make
#                                 else:
#                                     details['make'] = 'na'
                                
#                                 model = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
#                                 if model:
#                                     details['model'] = model.split(" ")[0]
#                                 else:
#                                     details['model'] = 'na'
                                
#                                 variant = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
#                                 if variant:
#                                     details['variant'] = " ".join(variant.split(" ")[1:])
#                                 else:
#                                     details['variant'] = 'na'
                                
#                                 loc =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="auction-location"]'))).text.strip()
#                                 if loc:
#                                     details['Center'] = loc
#                                 else:
#                                     details['Center'] = 'na'
                                    
#                             except Exception as e:
#                                 print("No veh title and location found", e)        

#                             # main agent --> done
#                             try:
#                                 agent = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]'))) 
#                                 if agent:
#                                     elements = agent.find_elements(By.TAG_NAME, "p")
#                                     if elements:
#                                         for ele in elements:
#                                             agent_lbl = ele.find_element(By.TAG_NAME, 'strong')
#                                             agent_val = ele.text.replace(agent_lbl.text, "").strip()
#                                             if agent_lbl and agent_val:
#                                                 details[agent_lbl.text.strip()] = agent_val
#                                             else:
#                                                 print("No agent data found")
#                                     else:
#                                         print("No agent elements")
                                    
#                                     lane =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]/p[3]')))     
#                                     if lane:
#                                         details['Lane'] = lane.text.strip().split(" ")[1]
#                                     else:
#                                         details['Lane'] = 'na'                             
#                             except Exception as e:
#                                 print(" No agent card")
                                
#                             # pricing --> done
#                             try:
#                                 pricing_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-list"]'))) 
#                                 if pricing_card:
#                                     pri_elements = pricing_card.find_elements(By.TAG_NAME, "p")
#                                     if pri_elements:
#                                         for pri_ele in pri_elements:
#                                             pri_lbl = pri_ele.find_element(By.TAG_NAME, 'strong')
#                                             pri_val = pri_ele.text.replace(pri_lbl.text, "").strip()
#                                             if pri_lbl and pri_val:
#                                                 details[pri_lbl.text.strip()] = pri_val
#                                             else:
#                                                 print("No pricing found")
#                             except Exception as e:
#                                 print(" No pricing card")
                                    
                            
#                             # pricing footer --> done
#                             try:
#                                 footer_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-footer"]'))) 
#                                 if footer_card:
#                                     fot_ele = footer_card.find_element(By.TAG_NAME, "p")
#                                     if fot_ele:
#                                         # for pri_ele in pri_elements:
#                                         fot_lbl = fot_ele.find_element(By.TAG_NAME, 'strong')
#                                         fot_val = fot_ele.text.replace(fot_lbl.text, "").strip()
#                                         if fot_lbl and fot_val:
#                                             details[fot_lbl.text.strip()] = fot_val
#                                         else:
#                                             print("No footer pricing found")
#                             except Exception as e:
#                                 print(" No footer pricing card")
                            
#                             # inspec rep --> done
#                             try:
#                                 inspec = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="auction-download"]'))).get_attribute("href")
#                                 if inspec:
#                                     details['Inspection Report'] = inspec
#                                 else:
#                                     details['Inspection Report'] = 'na'
#                             except Exception as e:
#                                 print("No inspec rep")                    
#                     except Exception as e:
#                         print("No main details card")
                        
#                     try:
#                         grade_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="gallery-main"]/img'))).get_attribute("src")
#                         if grade_main:
#                             grade = grade_main.split("/")[-1].split(".")[0]
#                             details['Grade'] = grade
#                         else:
#                             details['Grade'] = 'na'
#                     except Exception as e:
#                         print("no grade")                            
                        
                            
#                     # images --> done
#                     try:
#                         img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="ug-thumbs-strip-inner"]'))) 
#                         if img_card:
#                             imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
#                             if imgs:
#                                 imgs_lst = [img.get_attribute("src") for img in imgs]
#                                 details['Images'] = ", ".join(imgs_lst)
#                             else:
#                                 print("No images")
#                     except Exception as e:
#                         print(f"No image card {e}")
                        
                    
#                     # veh summary and inspection
#                     try:
#                         tabs_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@id="vehicleTab"]')))
#                         if tabs_card:
                            
#                             # veh summary --> done
#                             try:
#                                 veh_sum_btn =WebDriverWait(tabs_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="summary-tab"]'))) 
#                                 if veh_sum_btn:
#                                     driver.execute_script("arguments[0].scrollIntoView();", veh_sum_btn)
#                                     veh_sum_btn.click()
                                    
#                                     # veh summary
#                                     try:  
#                                         veh_cont_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="vehicleTabContent"]')))
#                                         if veh_cont_card:
#                                             # additional info --> done
#                                             try:
#                                                 add_info = veh_cont_card.find_element(By.XPATH, './/div[@id="vehicleTabContent"]//p').text.strip()
#                                                 if add_info:
#                                                     details['Additional Information'] = add_info
#                                                 else:
#                                                     details['Additional Information'] = 'na'
#                                             except Exception as e:
#                                                 print("No add info")
                                            
#                                             # other info --> done
#                                             try:       
#                                                 veh_sum = WebDriverWait(veh_cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-summary"]')))
#                                                 if veh_sum:
#                                                     veh_trs = WebDriverWait(veh_sum, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
#                                                     if veh_trs:
#                                                         for veh_tr in veh_trs:
#                                                             veh_lbl = veh_tr.find_element(By.TAG_NAME, 'td').text.strip()
#                                                             veh_val = veh_tr.find_element(By.TAG_NAME, 'strong').text.strip()
#                                                             if veh_lbl and veh_val:
#                                                                 details[veh_lbl] = veh_val
#                                                             else:
#                                                                 print("No veh details")
#                                             except Exception as e:
#                                                 print("No veh summary info")        
#                                         else:
#                                             print("No veh summary button")
#                                     except Exception as e:
#                                         print("No veh summary card found")
#                             except Exception as e:
#                                 print("No veh summary card")
                            
                            
#                             # inspection --> remains
#                             try:
#                                 inspec_btn =WebDriverWait(tabs_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="spec-tab"]'))) 
#                                 if inspec_btn:
#                                     driver.execute_script("arguments[0].scrollIntoView();", inspec_btn)
#                                     inspec_btn.click()
#                                     time.sleep(3)
                                    
#                                     # cond images
#                                     try:
#                                         cond_imgs_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="slick-track"])[2]')))
#                                         if cond_imgs_card:
#                                             cond_imgs = WebDriverWait(cond_imgs_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img')))
#                                             if cond_imgs:
#                                                 cond_imgs_lst = [cond_img.get_attribute("src") for cond_img in cond_imgs]
#                                                 details['Damaged_images'] = ", ".join(cond_imgs_lst)
#                                             else:
#                                                 details['Damaged_images'] = 'na'
#                                         else:
#                                             print("no cond image card")
#                                     except Exception as e:
#                                         print("no cond image card")
                                    
#                                     # interior and tyres
#                                     try:
#                                         int_tyre_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="row mb+"]')))
#                                         if int_tyre_card:
                                            
#                                             # interior
#                                             try:
#                                                 inspec_inter = WebDriverWait(int_tyre_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[1]/table/tbody')))
#                                                 if inspec_inter:
#                                                     inspec_trs = WebDriverWait(inspec_inter, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
#                                                     if inspec_trs:
#                                                         for inspec_tr in inspec_trs:
#                                                             inspec_lbl = inspec_tr.find_element(By.TAG_NAME, "th").text.strip()
#                                                             inspec_val = inspec_tr.find_element(By.TAG_NAME, "td").text.strip()
#                                                             if inspec_lbl and inspec_val:
#                                                                 details[inspec_lbl] = inspec_val
                
#                                                     else:
#                                                         print("No interior")
#                                             except Exception as e:
#                                                 print("No interior", e)     
                                            
#                                             # tyres
#                                             try:
#                                                 tyre_main = WebDriverWait(int_tyre_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]')))
#                                                 if tyre_main:
#                                                     # tyre = WebDriverWait(tyre_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]')))
#                                                     # interior
#                                                     try:
#                                                         tyre_inter = WebDriverWait(int_tyre_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody')))
#                                                         if tyre_inter:
#                                                             tyre_trs = WebDriverWait(tyre_inter, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
#                                                             if tyre_trs:
#                                                                 for tyre_tr in tyre_trs:
#                                                                     tyre_lbl = tyre_tr.find_elements(By.TAG_NAME, "td")
#                                                                     if tyre_lbl:
#                                                                         details[tyre_lbl[0].text.strip()] = [tyre_lbl[1].text.strip(), tyre_lbl[2].text.strip()]
                                                                        
#                                                                     # tyre_val = tyre_tr.find_element(By.TAG_NAME, "td").text.strip()
#                                                                     # if tyre_lbl and tyre_val:
                        
#                                                             else:
#                                                                 print("No interior")
#                                                     except Exception as e:
#                                                         print("No interior", e)
#                                             except Exception as e:
#                                                 print(f"No tyre data")
#                                     except Exception as e:
#                                         print("No interior and tyre")
#                             except Exception as e:
#                                 print(f"No isnpec tab")
                                
#                     except Exception as e:
#                         print("No buttons found")
                    
#                     car_count+=1
#                     results.append(details)
#                     driver.back()
        
#                 except Exception as e:
#                     print("No cars found")        

#             if car_count % 30 == 0: 
#                 try:
#                     next_link = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Load More"]')))
#                     if next_link:
#                         time.sleep(5)
#                         next_link.click()
                        
#                         WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
#                         print("Hit the next button")
#                 except Exception as e:
#                     print("No next found")
#                     break
     
#         except Exception as e:
#             print("No cars found")
#             break
        
#     df = pd.DataFrame.from_dict(results)
#     df.to_csv("data.csv", index=False)
#     driver.quit()
# path = "https://www.centralcarauctions.com/auction/776"
# scarpe(path)

In [22]:
df = pd.read_csv("data.csv")
df.rename(columns={"Order:":"Lot"}, inplace=True)
df.to_csv("demo.csv", index=False)
df = pd.read_csv("demo.csv")
type(df['Lot'])

EmptyDataError: No columns to parse from file

In [ ]:
df = pd.read_csv("data.csv")
df.rename(columns={"Sale date:":"Start Date", "Sale time:":"Start Time", "Order:":"Lot", "Cap Clean:":"Cap Clean", "Cap Avg.:":"Cap Average", "Cap Retail:":"Cap Retail", "Registration:":"Reg", "Reg year:":"D.O.R", "Autotrader Retail Value:":"Autotrader Retail Value", "Mileage:":"Mileage", "MOT:":"MOT Due", "Service History:":"Service History", "VAT:":"VAT Status", 'V5:':"V5", 'Owners:':"Former Keepers", 'Transmission:':"Transmission", 'Fuel:':"Fuel Type", 'Body:':"Body Type", 'Doors:':"Doors", 'Engine size:':"CC", 'Colour:':"Colour", "Service Stamps:":"No of services"}, inplace=True)
df['Mileage warranted']  = df['Mileage'].apply(lambda x : x.split("\n")[-1] if isinstance(x, str) and "\n" in x else None)
df['Mileage']  = df['Mileage'].apply(lambda x : x.split("\n")[0] if isinstance(x, str) and "\n" in x else None)
df['CC'] = df['CC'].apply(lambda x: f"{int(x.split("cc")[0])/1000:.1f}" if isinstance(x, str) else f"{x / 1000:.1f}")
df['Year'] = df['D.O.R'].apply(lambda x : x.split(" ")[-1] if isinstance(x, str) and " " in x else None)

info_cols = ['Carpet condition', 'Seat condition', 'Upholstery condition']
existing_cols = [col for col in info_cols if col in df.columns]
# Build the 'Additional Information' column
df['Additional Information'] = df[existing_cols].apply(lambda row: {col: row[col] if pd.notnull(row[col]) else 'na' for col in existing_cols},axis=1)
df.drop(columns=existing_cols + ['Lot number:'], inplace=True)
df = df.drop_duplicates(subset='Lot', keep='first')

df.to_csv("cca_data.csv", index=False)
# os.remove("data.csv")

### images and reports

In [ ]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from Central Car Auction"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [ ]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.
df = pd.read_csv("cca_data.csv")
reports = df[['Reg', 'Inspection Report']]
reg_img = df[['Reg', 'Images']]
cond_img = df[['Reg', 'Damaged_images']]

def download_reports(data, main_folder="Inspection Report"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        report_urls = row["Inspection Report"]
        
        # Check if the inspection report is missing
        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")
            # continue
            
        else:
            if not report_urls.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                report_urls = urljoin("https://", report_urls) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(report_urls)
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(report_urls, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"

                # Extracts the file extension (e.g., pdf). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}: {e}")

# images 
def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        if pd.isna(row["Images"]) or not isinstance(row["Images"], str) or row["Images"].strip() == "":
            print(f"Skipping {reg_no} due to missing images.")
        else:

            image_urls = row["Images"].split(", ")
            
            # Create a subfolder for the car registration number
            reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
            os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
            
            # loop around the urls of the current row and also save index for further use
            for idx, url in enumerate(image_urls):
                url = url.strip()  # Remove extra spaces
                if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                    url = urljoin("https://", url) # sets the urls starting from https://.....
                
                # parse the url
                parsed_url = urlparse(url)

                # check if the parsed url is valid
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                    continue
                
                # try downloading images
                try:
                    # Download the image
                    response = requests.get(url, stream=True) # send the url for downloading
                    response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                    
                    # If no file name is found, assigns a default name based on the index.
                    file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                    # Extracts the file extension (e.g., jpg). from the last index value
                    file_extension = file_name.split(".")[-1]
                    
                    # Ensure the file has a valid extension
                    if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                        file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                    
                    # Construct the full path for the file
                    full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                    
                    # Save the image
                    with open(full_file_name, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)

                    # Add watermark
                    add_watermark_to_image(full_file_name)
                    
                    print(f"Downloaded: {full_file_name}")
                except Exception as e:
                    print(f"Failed to download {url} for {reg_no}: {e}")

# cond_images 
def download_images_cond(data, main_folder="Damaged_images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        if pd.isna(row["Damaged_images"]) or not isinstance(row["Damaged_images"], str) or row["Damaged_images"].strip() == "":
            print(f"Skipping {reg_no} due to missing images.")
        else:

            image_urls = row["Damaged_images"].split(", ")
            
            # Create a subfolder for the car registration number
            reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
            os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
            
            # loop around the urls of the current row and also save index for further use
            for idx, url in enumerate(image_urls):
                url = url.strip()  # Remove extra spaces
                if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                    url = urljoin("https://", url) # sets the urls starting from https://.....
                
                # parse the url
                parsed_url = urlparse(url)

                # check if the parsed url is valid
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                    continue
                
                # try downloading images
                try:
                    # Download the image
                    response = requests.get(url, stream=True) # send the url for downloading
                    response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                    
                    # If no file name is found, assigns a default name based on the index.
                    file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                    # Extracts the file extension (e.g., jpg). from the last index value
                    file_extension = file_name.split(".")[-1]
                    
                    # Ensure the file has a valid extension
                    if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                        file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                    
                    # Construct the full path for the file
                    full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                    
                    # Save the image
                    with open(full_file_name, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)

                    # Add watermark
                    add_watermark_to_image(full_file_name)
                    
                    print(f"Downloaded: {full_file_name}")
                except Exception as e:
                    print(f"Failed to download {url} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_reports, args=(reports,))
    thread2 = threading.Thread(target=download_images, args=(reg_img,))
    thread3 = threading.Thread(target=download_images_cond, args=(cond_img,))

    thread1.start()
    thread2.start()
    thread3.start()
    
    
    thread1.join()
    thread2.join()
    thread3.join()

if __name__ =="__main__":
    start_funcs()


KeyError: "['Damaged_images'] not in index"